In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [2]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [ ]:
def Expected_SARSA(gridworld:GridWorld_v2.GridWorld_v2,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是Expected_SARSA算法

    Parameters:
    gamma (float): 近视远视程度
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5)) #初始化action_value table
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    epsilon = 0.5
    for episode in range(num_episodes):
        #清除输出，可以更好的展示策略
        # time.sleep(0.2)
        # clear_output(wait=True)
        
        print("episode",f"{episode}/{num_episodes}")
        if(epsilon > final_epsilon) :#随着episode增加，epsilon逐渐减小
            epsilon -= 0.001
        else:
            epsilon = final_epsilon

        # p1是目标方向的概率，p0是另外四个方向的概率
        p1 = 1-epsilon * (4/5)
        p0 = epsilon/5
        d = {1:p1, 0:p0}
        # policy_epsilon是policy取epsilon-greedy的概率决策
        print("p1",p1,"p0",p0)
        policy_epsilon = np.vectorize(d.get)(policy)

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        initState=10
        initAction=random.randint(0,4)  

        if trajectorySteps==-1:
            stop_when_reach_target = True
        Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                  action=initAction, 
                                                  policy=policy_epsilon, # policy_epsilon是policy取epsilon-greedy的概率决策
                                                  steps=trajectorySteps, 
                                                  stop_when_reach_target=True) #记录轨迹，与v1版本不同，这里的policy是一个 (rows*columns) * actions的二维列表，其中每一行的总和为1，代表每个state选择五个action的概率总和为1
        Trajectory.append((17,4,1,17,4)) #让他最后自转一下
        print("trajectorySteps",len(Trajectory))
        

        
        # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
        steps = len(Trajectory) - 1
        for k in range(steps,-1,-1):
            #State，Action，Reward，NextState，NextAction
            tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
            cnt[tmpstate] += 1
            #Expected SARSA,这里的v是一个期望，因为policy_epsilon是一个概率分布，所以v是一个期望
            #action_value[nextState]:
            # 这是一个Q值数组，具有当前状态为 nextState 的所有可能动作（actions）对应的Q值。数组的每个元素表示在 nextState 状态下选择某个特定动作的预期收益。
            # policy_epsilon[nextState]:
            # 这是我们当前策略（policy）在 nextState 状态下的概率分布。它是通过 epsilon-greedy 策略计算得出的，表示在这个状态下，每个可能动作被选择的概率。这里的v就是一个加权平均
            v = (action_value[nextState] * policy_epsilon[nextState]).sum() 
            TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * v) #TD-error
            action_value[tmpstate][tmpaction] -= learning_rate * TD_error #更新Q值

        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
        policy_epsilon = np.vectorize(d.get)(policy)  #epsilon-greedy的概率分布
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5)) 

        state_value = np.sum(policy_epsilon * action_value,axis=1)  #为了打印显示理解
        mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean() #为了打印显示理解
        
        gridworld.showPolicy(policy)
        print(np.round(state_value,decimals=4).reshape(5,5))
        print("mean_state_value", mean_state_value)

    return gridworld
    

In [4]:
Expected_SARSA(gridworld) #效果非常好，不仅局部最优，而且全局最优

episode 0/600
p1 0.6008 p0 0.0998
trajectorySteps 42
[[ 3  1  2  5  0]
 [ 6  3 15  1  0]
 [ 2  1  1  0  0]
 [ 0  0  2  0  0]
 [ 0  0  0  0  0]]
⬆️⬆️⬆️➡️⬆️
➡️⏫️⏩️⬆️⬆️
⬇️⬆️⏬⬆️⬆️
⬆️⏫️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
[[-0.0001 -0.001  -0.     -0.0003  0.    ]
 [-0.     -0.003  -0.0139 -0.001   0.    ]
 [-0.     -0.      0.0006  0.      0.    ]
 [ 0.      0.     -0.0004  0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]]
mean_state_value -0.000766009692857978
episode 1/600
p1 0.6015999999999999 p0 0.0996
trajectorySteps 144
[[24 33 13 12 29]
 [ 1  9  3  5  3]
 [ 2  4  1  1  0]
 [ 1  1  2  0  0]
 [ 0  0  0  0  0]]
⬇️➡️➡️🔄🔄
⬇️⏪⏫️➡️➡️
⬅️➡️⏬⬆️⬆️
⬆️⏩️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
[[-0.0018 -0.0079 -0.0016 -0.0004 -0.0022]
 [-0.001  -0.004  -0.0159 -0.001  -0.    ]
 [-0.     -0.002   0.0012 -0.001   0.    ]
 [-0.001  -0.      0.0008  0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]]
mean_state_value -0.0015092679263008831
episode 2/600
p1 0.6024 p0 0.0994
trajectorySteps 61
[[ 0  0  0  0  0]
 [ 6  1  0  0 